## Import modules

These are the bare minimum models required to run the code in this example notebook.  Feel free to add more if you modify this notebook (for example, if you want to manually make some custom plots, you would likely import matplotlib).

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import ALS

## Define/import user model and instantiate KineticModel object

In [2]:
from ex_model_1 import model_H2O2_depletion
model = ALS.KineticModel(model_H2O2_depletion)

## Load and organize species data

Step 1:

In [3]:
tbin = 10                    # tbin used when data was loaded from Igor
t_start = -20                # Trim the data to t_start inclusive (ms)
t_end = 59.8                 # Trim the data to t_end inclusive (ms)
fn_val = 'ex_data_val_1.csv' # Filename for the data values
fn_err = 'ex_data_err_1.csv' # Filename for the error in the data

dt = 0.02                    # Fundaemental kinetic time step (ms)
t = np.linspace(-20,130,num=int(150/(dt*tbin)),endpoint=False)

df_val = pd.read_csv(fn_val)
df_err = pd.read_csv(fn_err)

idx_start = np.abs(t-t_start).argmin()
idx_end = np.abs(t-t_end).argmin()
drop_indices = np.concatenate((np.arange(idx_start),np.arange(idx_end+1,t.size)))

t = t[idx_start:idx_end+1]
df_val = df_val.drop(drop_indices, axis=0).reset_index(drop=True)
df_err = df_err.drop(drop_indices, axis=0).reset_index(drop=True)

Step 2:

In [16]:
data = {}
data['H2O2'] = {'val':df_val['H2O2'].values, 'err':df_err['H2O2'].values, 'fit':True, 'weight':1}
data['HO2']  = {'val':df_val['HO2'].values,  'err':df_err['HO2'].values,  'fit':False, 'weight':1}
data['OH']   = {'val':df_val['OH'].values,   'err':df_err['OH'].values,   'fit':False, 'weight':1}
data = pd.DataFrame.from_dict(data, orient='index')
display(data)

val  \
H2O2  [6135.0, 6162.0, 6025.0, 6005.0, 6027.0, 6101....   
HO2   [-3.15, -4.15, -3.15, -4.15, 5.85, 6.85, -1.15...   
OH    [-0.82, 0.18, 0.18, -0.82, -0.82, -0.82, 1.18,...   

                                                    err    fit  weight  
H2O2  [78.326, 78.498, 77.621, 77.492, 77.634, 78.10...   True       1  
HO2   [5.0, 4.899, 5.0, 4.899, 5.831, 5.9161, 5.1962...  False       1  
OH    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.4142, 1.0, 1....  False       1

## Organize model and ALS parameters

In [5]:
c_H2O2_0 = 3.25e14
idx_zero = np.abs(t-(-1.68)).argmin()
S_H2O2 = df_val.iloc[:idx_zero,df_val.columns.get_loc('H2O2')].mean() / c_H2O2_0

model_params = {}
model_params['X0'] =         {'val':4.0e13,     'err':0,  'fit':True}
model_params['k_OH_wall'] =  {'val':200.,        'err':0, 'fit':True}
model_params['k_HO2_wall'] = {'val':100.,        'err':0, 'fit':True}
model_params['T'] =          {'val':298.,       'err':0,  'fit':False}
model_params['P'] =          {'val':30.,        'err':0,  'fit':False}
model_params['c_H2O2_0'] =   {'val':c_H2O2_0,   'err':0,  'fit':False}
model_params = pd.DataFrame.from_dict(model_params, orient='index')

print('Inputted Model Params:')
display(model_params)

ALS_params = {}
ALS_params['S_H2O2'] =  {'val':S_H2O2,    'err':0, 'fit':False}
ALS_params['S_OH'] =    {'val':5e-14,     'err':0, 'fit':True}
ALS_params['S_HO2'] =   {'val':5e-14,     'err':0, 'fit':True}
ALS_params['A'] =       {'val':-3.76e-1,  'err':0, 'fit':False}
ALS_params['B'] =       {'val':4.85e-3,   'err':0, 'fit':False}
ALS_params['t0'] =      {'val':-1.68,     'err':0, 'fit':False}
ALS_params = pd.DataFrame.from_dict(ALS_params, orient='index')

print('Inputted ALS Params:')
display(ALS_params)

Inputted Model Params:


val  err    fit
P           3.000000e+01    0  False
T           2.980000e+02    0  False
X0          4.000000e+13    0   True
c_H2O2_0    3.250000e+14    0  False
k_HO2_wall  1.000000e+02    0   True
k_OH_wall   2.000000e+02    0   True

Inputted ALS Params:


val  err    fit
A      -3.760000e-01    0  False
B       4.850000e-03    0  False
S_H2O2  1.857629e-11    0  False
S_HO2   5.000000e-14    0   True
S_OH    5.000000e-14    0   True
t0     -1.680000e+00    0  False

## Perform a fit

In [111]:
df_p, df_cov_p, df_corr_p, cost, mesg, ier \
    = model.fit(t, tbin, data, model_params, ALS_params, err_weight=True)

df_p.to_csv('params.csv')

Optimization terminated successfully.
Exit Code = 2
Exit Message = The relative error between two consecutive iterates is at most 0.000000

Optimized Cost Function Value = 9325.36

Optimized Parameters and Standard Errors:


val           err
X0          8.401581e+12  7.497760e+10
k_HO2_wall -2.029806e-01  2.709946e-03
k_OH_wall   1.274894e+02  1.419230e+00
S_HO2       2.045543e-10  7.717097e-13
S_OH        5.840777e-12  2.801142e-13


Correlation Matrix:


X0  k_HO2_wall  k_OH_wall     S_HO2      S_OH
X0          1.000000   -0.142125  -0.292723 -0.852149 -0.176975
k_HO2_wall -0.142125    1.000000  -0.530122 -0.022692  0.072416
k_OH_wall  -0.292723   -0.530122   1.000000  0.216808  0.039554
S_HO2      -0.852149   -0.022692   0.216808  1.000000  0.140795
S_OH       -0.176975    0.072416   0.039554  0.140795  1.000000

## Plot the data overlaid with the scaled model

Important: Despite the plot_fit name, this function does not actually perform a fit.


Use cases:
1. plotting a previous fit without having to rerun the fit code\n
2. playing around with initial guesses before performing a fit\n
3. saving optimized model * scale factor for each species in order to plot elsewere

In [24]:
% matplotlib notebook
model.plot_fit(t, tbin, data, model_params, ALS_params)

<IPython.core.display.Javascript object>

In [23]:
import importlib
importlib.reload(ALS)
model = ALS.KineticModel(model_H2O2_depletion)